In [1]:
# library imports

# file IO, processing
import pandas as pd

# lin alg
import numpy as np

# db engine
from config import get_engine

# linescores
from analysis import get_scores

In [5]:
# engine
database = "dnorm"
engine = get_engine(database=database)

In [9]:
# scores
scores = get_scores(classification='men 8 end')

In [10]:
scores.head()

,PivotedScoreID,GameID,EndNum,EndResult,Hammer,Score,TeamID,EventID,EventName,Year,EndMonth,EventDayEnd,Season,FinalScore,Team,EndSituation,OpponentTeamID,OpponentFinalScore,Win
0,751418,47846,1,-1,1,0,121243,5329,Red Deer Curling Classic,2018,11,19,2018-2019,6,Adam Norget,0,123213,4,1.0
1,751420,47846,2,3,1,0,121243,5329,Red Deer Curling Classic,2018,11,19,2018-2019,6,Adam Norget,-1,123213,4,1.0
2,751422,47846,3,-1,0,3,121243,5329,Red Deer Curling Classic,2018,11,19,2018-2019,6,Adam Norget,2,123213,4,1.0
3,751424,47846,4,0,1,3,121243,5329,Red Deer Curling Classic,2018,11,19,2018-2019,6,Adam Norget,1,123213,4,1.0
4,751426,47846,5,0,1,3,121243,5329,Red Deer Curling Classic,2018,11,19,2018-2019,6,Adam Norget,1,123213,4,1.0


In [50]:
# helper functions


def remove_blanks(srs):
    return srs[np.where(srs!=0,True,False)]

def hammer_conversion(srs):
    srs_no_blanks = remove_blanks(srs)
    if len(srs_no_blanks) == 0:
        return 0
    return np.sum(np.where(srs_no_blanks >= 2,1,0)) / len(srs_no_blanks)

def force_efficiency(srs):
    srs_no_blanks_steals = srs[np.where(srs<0,True,False)]
    if len(srs_no_blanks_steals)==0:
        return 0
    return np.sum(np.where(srs_no_blanks_steals == -1,1,0))/len(srs_no_blanks_steals)

def steal_efficiency(srs):
    if len(srs) == 0:
        return 0
    return np.sum(np.where(srs>0,1,0))/len(srs)

def steal_defence(srs):
    if len(srs) == 0:
        return 0
    return np.sum(np.where(srs<0,0,1))/len(srs)

In [51]:

# misc tests

assert hammer_conversion(np.array([1,1,1])) == 0
assert hammer_conversion(np.array([])) == 0
assert hammer_conversion(np.array([0,0,0])) == 0
assert hammer_conversion(np.array([1,1,1,1,0,2])) == 0.2
assert hammer_conversion(np.array([0,0,0,2])) == 1
assert hammer_conversion(np.array([-1,1,1,1,0,2])) == 0.2

In [52]:
assert force_efficiency(np.array([1,1,1])) == 0
assert force_efficiency(np.array([])) == 0
assert force_efficiency(np.array([0,0,0])) == 0
assert force_efficiency(np.array([-1,-2])) == 0.5
assert force_efficiency(np.array([-1,1])) == 1
assert force_efficiency(np.array([-1,-2,1])) == 0.5
assert force_efficiency(np.array([-1,-2,1,0])) == 0.5
assert force_efficiency(np.array([-1,-2,1,-1])) == 2/3

In [53]:
assert steal_efficiency(np.array([1,1,1])) == 1
assert steal_efficiency(np.array([])) == 0
assert steal_efficiency(np.array([0,0,0])) == 0
assert steal_efficiency(np.array([-1,-2])) == 0
assert steal_efficiency(np.array([-1,1])) == .5
assert steal_efficiency(np.array([-1,-2,1])) == 1/3
assert steal_efficiency(np.array([-1,-2,1,0])) == 1/4
assert steal_efficiency(np.array([-1,-2,1,-1])) == 1/4

In [54]:
assert steal_defence(np.array([1,1,1])) == 1
assert steal_defence(np.array([])) == 0
assert steal_defence(np.array([0,0,0])) == 1
assert steal_defence(np.array([-1,-2])) == 0
assert steal_defence(np.array([-1,1])) == .5
assert steal_defence(np.array([-1,-2,1])) == 1/3
assert steal_defence(np.array([-1,-2,1,0])) == 1/2
assert steal_defence(np.array([-1,-2,1,-1])) == 1/4

In [59]:
test_scores=scores.loc[scores.EventID==6815].reset_index(drop=True)

In [64]:
test_scores.groupby(['Team','Hammer'])[['EndResult']].aggregate([hammer_conversion,'count'])

EndResult      
                     hammer_conversion count
Team          Hammer                        
Beth Farmer   0               0.050000    21
              1               0.333333    22
Bruce Mouat   0               0.120000    29
              1               0.666667    21
Cameron Bryce 0               0.120000    30
              1               0.315789    21
Eve Muirhead  0               0.166667    20
              1               0.263158    20
Fay Henderson 0               0.000000    14
              1               0.130435    24
Gina Aitken   0               0.045455    23
              1               0.423077    28
James Craik   0               0.208333    27
              1               0.363636    27
Maggie Wilson 0               0.105263    20
              1               0.235294    21
Ross Paterson 0               0.000000    25
              1               0.478261    26
Ross Whyte    0               0.107143    32
              1               0.370370    31

In [69]:
hammer_conversion(test_scores.loc[(test_scores.Team=='Bruce Mouat') & (test_scores.Hammer==1)].EndResult)

0.6666666666666666